In [1]:
import json
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import random
import shutil

from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor, default_argument_parser, default_setup
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog

from deepent.data.register_datasets import register_datasets
from deepent.config import add_deepent_config
from tools.vis import visualize_single
from tools.train_net import *
from tools.predictor import RGBDPredictor

In [2]:
config_file = '../configs/deepent_fuse_rcnn_R_50_FPN.yaml'
threshold = 0.5
model = '../output/baseline_fuse_07_02_2020/model_0089999.pth'
samples = 1
dataset = 'CPT2a-n_train'
type_ = 'many'
opts = []

class Cfg_Args:
    def __init__(self, conf, tr, mod, sam, ty, opt, dts):
        self.config_file = conf
        self.threshold = tr
        self.model = mod
        self.samples = sam
        self.dataset = dts
        self.type = ty
        self.opts = []
        self.output = None
        
args = Cfg_Args(config_file, threshold, model, samples, type_, opts, dataset)

In [3]:
cfg = get_cfg()
add_deepent_config(cfg)
cfg.merge_from_file(config_file)
cfg.merge_from_list(opts)
cfg.MODEL.WEIGHTS = model 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold 
cfg.freeze()
default_setup(cfg, args)

[03/19 01:30:16 detectron2]: Rank of current process: 0. World size: 1
[03/19 01:30:17 detectron2]: Environment info:
------------------------  --------------------------------------------------------------------------------------
sys.platform              linux
Python                    3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) [GCC 7.2.0]
numpy                     1.15.4
detectron2                0.1.1 @/home/ubuntu/drone-tree-id/lib/detectron2/detectron2
detectron2 compiler       GCC 5.4
detectron2 CUDA compiler  10.1
detectron2 arch flags     sm_75
DETECTRON2_ENV_MODULE     <not set>
PyTorch                   1.4.0 @/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch
PyTorch debug build       False
CUDA available            True
GPU 0                     Tesla T4
CUDA_HOME                 /usr/local/cuda-10.1
NVCC                      Cuda compilation tools, release 10.1, V10.1.243
Pillow                    5.4.1
torchvision               0.5.0 @

[03/19 01:30:17 detectron2]: Full config saved to ./output/config.yaml


In [4]:
arg_array = ['--eval-only',
             '--config-file', '/home/ubuntu/drone-tree-id/configs/deepent_fuse_rcnn_R_50_FPN.yaml',
             'OUTPUT_DIR',' home/ubuntu/drone-tree-id/output/baseline_fuse_07_02_2020/ ',
             'MODEL.WEIGHTS', '/home/ubuntu/drone-tree-id/output/baseline_fuse_07_02_2020/model_final.pth']

In [5]:
def ind_eval(args):
    #create new mini-datasets
    test_tiles = []
    test_tile_segs = []
    for location in glob.glob('/home/ubuntu/RGBD-Tree-Segs-Clean/test/*'):
        filenames = glob.glob(location + '/*.png')
        with open(os.path.join(location,'segs.json'),'r') as segfile:
            segs = json.load(segfile)
            for ind in np.random.randint(0,len(filenames),2):
                test_tiles.append(filenames[ind])
                test_tile_segs.append(list(filter(lambda x : x["file_name"]==filenames[ind],segs)))
                
    test_dir = '/home/ubuntu/RGBD-Tree-Segs-Clean/test/'                
    folder_names = []                 
    for i,path in enumerate(test_tiles):
        folder_names.append('temporary_'+str(i))
        os.mkdir(test_dir + folder_names[-1])
        shutil.copy(path, test_dir + folder_names[-1] + '/' + path.split('/')[-1])
        with open(os.path.join(test_dir,folder_names[-1],'segs.json'),'w') as segfile:
            segfile.write(json.dumps(test_tile_segs[i]))
    test_set_names = list(map(lambda x : x + "_test", folder_names))        
    register_datasets(f'/home/ubuntu/RGBD-Tree-Segs-Clean/')
    
    cfg = get_cfg()
    add_deepent_config(cfg)
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    cfg.DATASETS.TEST = test_set_names
    cfg.freeze()
    default_setup(cfg, args)
    
    #Run Eval
    model = Trainer.build_model(cfg)
    DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
        cfg.MODEL.WEIGHTS, resume=args.resume
    )
    res = Trainer.test(cfg, model,wandb_on=False)
    if comm.is_main_process():
                verify_results(cfg, res)
        
    #Visualize with different models and show evals
    predictor = RGBDPredictor(cfg)
    fig,axes = plt.subplots(10,1,figsize=(70,70))
    ax = axes.ravel()
    for i,test_set_name in enumerate(test_set_names):
        data = list(DatasetCatalog.get(test_set_name))
        metadata = MetadataCatalog.get(test_set_name)
        dic=data[0]
        rgba = cv2.imread(dic["file_name"], cv2.IMREAD_UNCHANGED)
        img = cv2.imread(dic["file_name"])
        predictions = predictor(rgba)
        visualizer = Visualizer(img, metadata=metadata)
        vis = visualizer.draw_instance_predictions(predictions["instances"].to("cpu")).get_image()
        name = res[test_set_name]["segm"]["AP"]
        ax[i].set_title("AP: " + str(name))
        ax[i].axis('off')
        ax[i].imshow(vis)
        
    #remove temporary dirs
    os.chdir('/home/ubuntu/RGBD-Tree-Segs-Clean/test')
    for folder in glob.glob('temporary*'):
        for file in glob.glob(folder + '/*'):
            os.remove(file)
        os.rmdir(folder)   
        
    return res

args = default_argument_parser().parse_args(arg_array)
results = ind_eval(args)

Registering Kelowna_train
Registering AMECT9_train
Registering SALCT1_train
Registering CPT2b_train
Registering CPT2a-n_train
Registering temporary_0_test
Registering temporary_4_test
Registering temporary_8_test
Registering temporary_7_test
Registering Kelowna_test
Registering AMECT9_test
Registering temporary_2_test
Registering SALCT1_test
Registering CPT2b_test
Registering temporary_1_test
Registering temporary_9_test
Registering temporary_6_test
Registering temporary_5_test
Registering temporary_3_test
Registering CPT2a-n_test
[03/19 01:30:27 detectron2]: Rank of current process: 0. World size: 1
[03/19 01:30:27 detectron2]: Rank of current process: 0. World size: 1
[03/19 01:30:27 detectron2]: Environment info:
------------------------  --------------------------------------------------------------------------------------
sys.platform              linux
Python                    3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) [GCC 7.2.0]
numpy                     1.15.4
de

[03/19 01:30:27 detectron2]: Running with full config:
CUDNN_BENCHMARK: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  FILTER_EMPTY_ANNOTATIONS: True
  NUM_WORKERS: 4
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: ()
  PROPOSAL_FILES_TRAIN: ()
  TEST: ['temporary_0_test', 'temporary_1_test', 'temporary_2_test', 'temporary_3_test', 'temporary_4_test', 'temporary_5_test', 'temporary_6_test', 'temporary_7_test', 'temporary_8_test', 'temporary_9_test']
  TRAIN: ('Kelowna_train', 'CPT2a-n_train', 'CPT2b_train', 'SALCT1_train', 'AMECT9_train')
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: False
    SIZE: [0.9, 0.9]
    TYPE: relative_range
  FORMAT: RGBA
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (600, 640, 672, 704, 736, 768, 800)
  MIN_SIZE_TRAIN_SAMPLING: choice
MODEL:
  ANCHOR_GENERATOR:
    ANGLES: 

[03/19 01:30:31 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): FusedResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
     

[03/19 01:30:31 fvcore.common.checkpoint]: Loading checkpoint from /home/ubuntu/drone-tree-id/output/baseline_fuse_07_02_2020/model_final.pth
[03/19 01:30:31 fvcore.common.checkpoint]: Loading checkpoint from /home/ubuntu/drone-tree-id/output/baseline_fuse_07_02_2020/model_final.pth
[03/19 01:30:31 fvcore.common.checkpoint]: Some model parameters are not in the checkpoint:
  backbone.bottom_up.fuse_res2.fuse_out.{weight, bias}
  backbone.bottom_up.fuse_res3.fuse_out.{weight, bias}
  backbone.bottom_up.fuse_res4.fuse_out.{weight, bias}
  backbone.bottom_up.fuse_res5.fuse_out.{weight, bias}
[03/19 01:30:31 fvcore.common.checkpoint]: Some model parameters are not in the checkpoint:
  backbone.bottom_up.fuse_res2.fuse_out.{weight, bias}
  backbone.bottom_up.fuse_res3.fuse_out.{weight, bias}
  backbone.bottom_up.fuse_res4.fuse_out.{weight, bias}
  backbone.bottom_up.fuse_res5.fuse_out.{weight, bias}
[03/19 01:30:31 d2.data.build]: Distribution of instances among all 3 categories:
|  categor

[03/19 01:30:32 d2.evaluation.coco_evaluation]: Evaluation results for segm: 
|  AP   |  AP50  |  AP75  |  APs  |  APm  |  APl  |
|:-----:|:------:|:------:|:-----:|:-----:|:-----:|
| 2.209 | 11.985 | 0.391  | 0.000 | 0.000 | 3.999 |
[03/19 01:30:32 d2.evaluation.coco_evaluation]: Per-category segm AP: 
| category   | AP    | category   | AP   | category   | AP   |
|:-----------|:------|:-----------|:-----|:-----------|:-----|
| coniferous | 2.209 | dead       | nan  | deciduous  | nan  |
[03/19 01:30:32 d2.evaluation.coco_evaluation]: Per-category segm AP: 
| category   | AP    | category   | AP   | category   | AP   |
|:-----------|:------|:-----------|:-----|:-----------|:-----|
| coniferous | 2.209 | dead       | nan  | deciduous  | nan  |
[03/19 01:30:32 d2.evaluation.testing]: copypaste: Task: bbox
[03/19 01:30:32 d2.evaluation.testing]: copypaste: Task: bbox
[03/19 01:30:32 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[03/19 01:30:32 d2.evaluation.testing]: copypa

...

[03/19 01:30:35 d2.evaluation.evaluator]: Start inference on 1 images
[03/19 01:30:35 d2.evaluation.evaluator]: Start inference on 1 images
[03/19 01:30:35 d2.evaluation.evaluator]: Total inference time: 0:00:00.241139 (0.241139 s / img per device, on 1 devices)
[03/19 01:30:35 d2.evaluation.evaluator]: Total inference time: 0:00:00.241139 (0.241139 s / img per device, on 1 devices)
[03/19 01:30:35 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:00 (0.160586 s / img per device, on 1 devices)
[03/19 01:30:35 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:00 (0.160586 s / img per device, on 1 devices)
[03/19 01:30:35 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[03/19 01:30:35 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[03/19 01:30:35 d2.evaluation.coco_evaluation]: Saving results to  home/ubuntu/drone-tree-id/output/baseline_fuse_07_02_2020/ /inference/coco_instances_results.json
[03/19 01:30:35 d2.

[03/19 01:30:35 d2.data.common]: Serialized dataset takes 0.02 MiB
WARNING [03/19 01:30:35 d2.evaluation.coco_evaluation]: json_file was not found in MetaDataCatalog for 'temporary_7_test'. Trying to convert it to COCO format ...
WARNING [03/19 01:30:35 d2.evaluation.coco_evaluation]: json_file was not found in MetaDataCatalog for 'temporary_7_test'. Trying to convert it to COCO format ...
WARNING [03/19 01:30:35 d2.data.datasets.coco]: Using previously cached COCO format annotations at ' home/ubuntu/drone-tree-id/output/baseline_fuse_07_02_2020/ /inference/temporary_7_test_coco_format.json'. You need to clear the cache file if your dataset has been modified.
WARNING [03/19 01:30:35 d2.data.datasets.coco]: Using previously cached COCO format annotations at ' home/ubuntu/drone-tree-id/output/baseline_fuse_07_02_2020/ /inference/temporary_7_test_coco_format.json'. You need to clear the cache file if your dataset has been modified.
[03/19 01:30:35 d2.evaluation.evaluator]: Start inference

[03/19 01:30:36 d2.data.build]: Distribution of instances among all 3 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
| coniferous | 24           |    dead    | 0            | deciduous  | 0            |
|            |              |            |              |            |              |
|   total    | 24           |            |              |            |              |
[03/19 01:30:36 d2.data.build]: Distribution of instances among all 3 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
| coniferous | 24           |    dead    | 0            | deciduous  | 0            |
|            |              |            |              |            |              |
|   total    | 24           |            |              |     

[03/19 01:30:37 d2.evaluation.coco_evaluation]: Per-category segm AP: 
| category   | AP    | category   | AP   | category   | AP   |
|:-----------|:------|:-----------|:-----|:-----------|:-----|
| coniferous | 0.043 | dead       | nan  | deciduous  | nan  |
[03/19 01:30:37 d2.evaluation.testing]: copypaste: Task: bbox
[03/19 01:30:37 d2.evaluation.testing]: copypaste: Task: bbox
[03/19 01:30:37 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[03/19 01:30:37 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[03/19 01:30:37 d2.evaluation.testing]: copypaste: 0.0646,0.2152,0.0000,nan,0.4950,0.0000
[03/19 01:30:37 d2.evaluation.testing]: copypaste: 0.0646,0.2152,0.0000,nan,0.4950,0.0000
[03/19 01:30:37 d2.evaluation.testing]: copypaste: Task: segm
[03/19 01:30:37 d2.evaluation.testing]: copypaste: Task: segm
[03/19 01:30:37 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[03/19 01:30:37 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[03/19 0

[03/19 01:30:37 d2.evaluation.coco_evaluation]: Evaluation results for segm: 
|  AP   |  AP50  |  AP75  |  APs  |  APm  |  APl  |
|:-----:|:------:|:------:|:-----:|:-----:|:-----:|
| 0.025 | 0.083  | 0.000  | 0.000 | 0.495 | 0.000 |
[03/19 01:30:37 d2.evaluation.coco_evaluation]: Per-category segm AP: 
| category   | AP    | category   | AP    | category   | AP    |
|:-----------|:------|:-----------|:------|:-----------|:------|
| coniferous | 0.074 | dead       | 0.000 | deciduous  | 0.000 |
[03/19 01:30:37 d2.evaluation.coco_evaluation]: Per-category segm AP: 
| category   | AP    | category   | AP    | category   | AP    |
|:-----------|:------|:-----------|:------|:-----------|:------|
| coniferous | 0.074 | dead       | 0.000 | deciduous  | 0.000 |
[03/19 01:30:37 d2.evaluation.testing]: copypaste: Task: bbox
[03/19 01:30:37 d2.evaluation.testing]: copypaste: Task: bbox
[03/19 01:30:37 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[03/19 01:30:37 d2.evaluation.test

In [ ]:
print("hi" + results["temporary_0_test"]["segm"]["AP"])

In [ ]:
#remove temporary dirs
os.chdir('/home/ubuntu/RGBD-Tree-Segs-Clean/test')
for folder in glob.glob('temporary*'):
    for file in glob.glob(folder + '/*'):
        os.remove(file)
    os.rmdir(folder)   